# 构建检索问答链

在 `C3 搭建数据库` 章节，我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [1]:
import sys
sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma

从环境变量中加载你的 API_KEY

In [2]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

加载向量数据库，其中包含了 ../../data_base/knowledge_db 下多个文档的 Embedding

In [5]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)

In [6]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：20


我们可以测试一下加载的向量数据库，使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

> ⚠️使用相似性搜索前，请确保你已安装了 OpenAI 开源的快速分词工具 tiktoken 包：`pip install tiktoken`

In [7]:
question = "什么是prompt engineering?"
docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


打印一下检索到的内容

In [8]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 3
1.4.1
式(1.1) 和式(1.2) 的解释. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
4
第2 章模型评估与选择
5
2.1
经验误差与过拟合
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
5
2.2
评估方法
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
5
2.2.1
算法参数（超参数）与模型参数. . . . . . . . . . . . . . . . . . . . . . . . . . . . .
6
2.2.2
验证集. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
6
2.3
性能度量
-----------------------------------------------------
检索到的第1个内容: 
 39
4.5.1
图(4.10) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
39
4.5.2
图(4.11) 的解释
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
39
第5 章神经网络
41
5.1
神经元模型. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
41
5.2
感知机与多层网络
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
41
5.2.1
式(5.1) 和式(5.2) 的推导. . . .

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [15]:
from zhipuai_llm import ZhipuAILLM
zhipuai_model = ZhipuAILLM(model="GLM-4-0520", temperature=0.01, api_key=zhipuai_api_key)
llm = zhipuai_model

In [16]:
llm.invoke("你好，请你自我介绍一下！")

'你好，我是智谱清言，是智谱 AI 公司训练的语言模型。我的目标是通过回答用户提出的问题来帮助他们解决问题。由于我是一个计算机程序，所以我没有自我意识，也不能像人类一样感知世界。我只能通过分析我所学到的信息来回答问题。'

In [19]:
import os 
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [20]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

llm.invoke("请你自我介绍一下自己！")

AIMessage(content='你好，我是一个智能助手，专门为用户提供各种服务和帮助。我可以回答问题、提供信息、解决问题等等。如果您有任何需要，请随时告诉我，我会尽力帮助您的。感谢您的使用！', response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 20, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

## 3. 构建检索问答链

In [14]:
from langchain.prompts import PromptTemplate

template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


再创建一个基于模板的检索链：

In [17]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


创建检索 QA 链的方法 RetrievalQA.from_chain_type() 有如下参数：
- llm：指定使用的 LLM
- 指定 chain type : RetrievalQA.from_chain_type(chain_type="map_reduce")，也可以利用load_qa_chain()方法指定chain type。
- 自定义 prompt ：通过在RetrievalQA.from_chain_type()方法中，指定chain_type_kwargs参数，而该参数：chain_type_kwargs = {"prompt": PROMPT}
- 返回源文档：通过RetrievalQA.from_chain_type()方法中指定：return_source_documents=True参数；也可以使用RetrievalQAWithSourceChain()方法，返回源文档的引用（坐标或者叫主键、索引）

## 4.检索问答链效果测试

In [18]:
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [20]:
result = qa_chain.invoke({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

大模型+知识库后回答 question_1 的结果：
南瓜书是对周志华教授的《机器学习》（西瓜书）中复杂公式进行解析和补充推导细节的辅助材料。它旨在帮助读者更好地理解西瓜书中的内容。谢谢你的提问！


In [21]:
result = qa_chain.invoke({"query": question_2})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

大模型+知识库后回答 question_2 的结果：
王阳明是中国明代著名的哲学家、教育家、军事家，被尊称为“阳明先生”。他的哲学思想被称为阳明心学，主张“知行合一”，影响深远。谢谢你的提问！


### 4.2 大模型自己回答的效果

In [22]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
llm.invoke(prompt_template)

/root/miniconda3/envs/llm-universe/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'南瓜书是对周志华教授的《机器学习》（通常称为西瓜书）中出现的公式进行详细解析和推导的辅助教材，其正式名称为《机器学习公式详解指南：南瓜书》。它由开源组织Datawhale的成员发起并编写，是对西瓜书中250个重难点公式进行详细解析和推导的集锦，旨在帮助读者理解和解决在学习机器学习过程中遇到的数学难题。\n\n南瓜书的内容以西瓜书中的知识点为基础，对西瓜书中没有详述的公式推导细节进行了补充和解释，非常适合那些希望深入了解机器学习算法背后数学原理的读者。由于其详尽的解析和推导，南瓜书被许多读者视为西瓜书的伴侣书，可以帮助读者“知其然，并知其所以然”。\n\n自2019年南瓜书在GitHub上发布以来，它受到了广泛的关注和好评，并在GitHub Trending上取得了很高的排名，获得了大量的star。随后，南瓜书也正式出版，为更多需要深入学习机器学习的读者提供了便利。'

In [23]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
llm.invoke(prompt_template)

'王阳明（1472年－1529年），名守仁，字伯安，号阳明，浙江绍兴人，是中国明代著名的哲学家、教育家、军事家、文学家。他是儒家心学的集大成者，提出了“知行合一”、“致良知”等思想，对后世影响深远。王阳明在思想、军事、政治等多个领域均有卓越成就，被后世尊为“千古完人”。他的一生充满波折，曾因直言进谏被贬至贵州龙场，但在那里他得以悟道，创立了阳明心学。王阳明的事迹和思想至今仍受到人们的推崇和研究。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


## 1. 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [24]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

## 2. 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

![](../../figures/Modular_components.png)

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习到关于提示工程的知识吗？”，并查看回答。

In [25]:
from langchain.chains import ConversationalRetrievalChain

retriever=vectordb.as_retriever()

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)
question = "我可以学习到关于提示工程的知识吗？"
result = qa.invoke({"question": question})
print(result['answer'])

我不确定你能否通过上述提供的材料学习到关于提示工程的知识。上述内容主要涉及的是机器学习相关的知识，特别是《机器学习》这本书（被称为西瓜书）以及作为补充的南瓜书，它们主要关注的是机器学习理论、公式推导等。提示工程通常指的是创建和应用提示来引导用户输入或操作，在人工智能助手和聊天机器人中应用较多，它更侧重于人机交互和自然语言处理。所以，除非西瓜书或南瓜书中有涉及这方面的内容，否则你可能需要寻找其他资源来学习提示工程。


然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [26]:
question = "为什么这门课需要教这方面的知识？"
result = qa({"question": question})
print(result['answer'])

在提示工程这门课程中需要教授关于人机交互和自然语言处理的知识，因为提示工程通常涉及到构建与用户进行交互的系统，这些系统需要能够理解和生成自然语言。人机交互的知识有助于设计出更符合用户使用习惯、更直观和高效的用户界面；自然语言处理则确保系统能够理解和生成自然语言文本，从而与用户进行流畅的沟通。这两方面的知识对于创建有效的提示和提升用户体验至关重要。


可以看到，LLM 它准确地判断了这方面的知识，指代内容是强化学习的知识，也就
是我们成功地传递给了它历史信息。这种持续学习和关联前后问题的能力，可大大增强问答系统的连续
性和智能水平。